# Using Personalize campaigns for making real-time predictions
This notebook exercises the campaign that have been built in the other notebooks. 


<a id='contents' />

## Content Table

1. [Loading libraries and data](#loading)
2. [Exercise the hrnn-metadata campaign](#campaign)
2. [Use real time events](#realtime)

<a id='loading' />

## Loading libraries and data
[(back to top)](#contents)

In [2]:
account_num = '<YOUR_ACCOUNT_NUMBER>'
import json
import boto3
import time
import datetime
import pandas as pd
from sklearn.utils import shuffle
import uuid

region   = boto3.Session().region_name 
print(region)

#The dataset group name defined in previous notebooks
dataset_group_name = 'video-dataset-group'
dg_arn = 'arn:aws:personalize:{}:{}:dataset-group/{}'.format(region, 
                                                             account_num, 
                                                             dataset_group_name)

#The interactions filename
interactions_filename = 'movies-interactions.csv'

#The ARN of the hrnn-metadata-hpo campaign, created in the previous notebook
hrnn_metadata_hpo_arn   = 'arn:aws:personalize:{}:{}:campaign/video-hrnn-metadata-hpo'.format(region, account_num)

us-east-1


In [3]:
personalize           = boto3.client('personalize')
personalize_runtime   = boto3.client('personalize-runtime')
personalize_events    = boto3.client('personalize-events')

Function to convert timestamp to datetime in string format:

In [6]:
def date_to_string(ts):
    return datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')

Read the data

In [8]:
int_expanded_df = pd.read_csv(interactions_filename)
int_expanded_df['WHEN'] = int_expanded_df['TIMESTAMP'].apply(date_to_string)

<a id='campaign' />

## Exercise the hrnn-metadata campaign
[(back to top)](#contents)

Here we try out the hrnn-metadata campaign. 
We ask Personalize for recommendations for a particular user. 

**Definition of some important functions**

Retrieve the previous history of revies an item, given the item ID and the interactions dataset

In [4]:
def show_item_interaction_history(int_df, item_id):
    _tmp_df = int_df[int_df.ITEM_ID == item_id].sort_values('TIMESTAMP')
    print('Interactions: ',len(_tmp_df))
    return _tmp_df[['USER_ID','ITEM_ID','WHEN']]

Retrieve the previous history of a user, given the user ID and the interactions dataset

In [5]:
def show_user_interaction_history(int_df, user_id):
    _tmp_df = int_df[int_df.USER_ID == int(user_id)].sort_values('TIMESTAMP')
    print('Interactions: ',len(_tmp_df))
    return _tmp_df[['USER_ID','ITEM_ID','WHEN']]

Verify if the campaign is active given the ARN

In [7]:
def is_campaign_active(c):
    _is_active = False
    
    try:
        _resp = personalize.describe_campaign(campaignArn = c)
        _campaign_status = _resp['campaign']['status']
        if _campaign_status == 'ACTIVE':
            _is_active = True
    except Exception as e:
        pass
        
    return _is_active

Print an item in the following format:

In [10]:
def print_item(item_id):
    print('Item: {}'.format(item_id))

**Define a sample of users to get recommendations for:**

In [12]:
users_to_try = int_expanded_df.sample(10)
users_to_try[['USER_ID']].head(3)

,USER_ID
1276031,24765170
1839317,37197598
1409884,27598168


Example of interaction history for 3 of the users:

In [13]:
show_user_interaction_history(int_expanded_df, 24765170)

Interactions:  3


,USER_ID,ITEM_ID,WHEN
1276030,24765170,Extract,2015-02-20 00:00:00
1276031,24765170,The Cheyenne Social Club,2015-02-20 00:00:00
1276032,24765170,The Sons of Katie Elder,2015-02-20 00:00:00


In [14]:
show_user_interaction_history(int_expanded_df, 37197598)

Interactions:  2


,USER_ID,ITEM_ID,WHEN
1839316,37197598,A Lamp In The Dark - The Untold History of the...,2012-11-06 00:00:00
1839317,37197598,Theodore Roosevelt: A Cowboy's Ride to the Whi...,2012-11-23 00:00:00


In [65]:
show_user_interaction_history(int_expanded_df, 27598168)

Interactions:  3


,USER_ID,ITEM_ID,WHEN
1409884,27598168,Monsters,2014-04-29 00:00:00
1409885,27598168,Quigley Down Under,2014-07-18 00:00:00
1409883,27598168,Cast Away,2014-11-17 00:00:00


Getting recommendations for these 10 users:

In [17]:
if is_campaign_active(hrnn_metadata_hpo_arn):
    for i in range(10):
        user_id     = str(users_to_try.iloc[i]['USER_ID'])

        print('Getting recommendations for user: {}'.format(user_id))
        response = personalize_runtime.get_recommendations(campaignArn=hrnn_metadata_hpo_arn, 
                                                           userId=user_id, 
                                                           numResults=10)

        items = response['itemList']
        for item in items:
            _curr_item_id  = item['itemId']
            print_item(_curr_item_id)
        print('')
else:
    print('HRNN-metadata campaign not active: {}'.format(hrnn_metadata_arn))

Getting recommendations for user: 24765170
Item: In Harm's Way
Item: Hondo
Item: Stand Up Guys
Item: Pilot
Item: Patriot Games
Item: Skyfall
Item: The Last Stand
Item: How To Steal A Million
Item: Flight
Item: Tucker: The Man and His Dream

Getting recommendations for user: 37197598
Item: Pilot
Item: Fat Man and Little Boy
Item: Star Trek: Nemesis
Item: Broken Arrow (1950)
Item: The Russian German War - Part 1
Item: Rommel - The Complete First Season
Item: Beverly Hills Cop III
Item: Force 10 from Navarone
Item: When Worlds Collide
Item: The Enemy Below

Getting recommendations for user: 27598168
Item: Jack Reacher
Item: Sherlock Holmes (2009)
Item: Pilot
Item: World War Z
Item: Star Trek Into Darkness (Plus Bonus Features)
Item: Skyfall
Item: Marvel's The Avengers
Item: The Last Stand
Item: The Hunger Games: Catching Fire
Item: The Magnificent Seven

Getting recommendations for user: 11195838
Item: Pilot
Item: The Ouija Experiment
Item: JESSABELLE
Item: "As Above, So Below"
Item: The 

<a id='realtime' />

## Use real time events
[(back to top)](#contents)

Here we use the event tracker mechanism of personalize to add some events on the fly after deployment of 
a campaign. We then show the impact on the recommendations, demonstrating that Personalize is able to 
react to changing user preferences on the fly.

In [18]:
#Name of the Personalize event Tracker we will create
tracker_name='VideoReviewTracker'

Function for verifying if the tracker is already active:

In [19]:
def is_tracker_active(tracker_name):
    _is_active = False
    _event_tracker_arn = ''
    _tracking_id = ''

    resp = personalize.list_event_trackers()
    trackers = resp['eventTrackers']

    for t in trackers:
        if t['name'] == tracker_name:
            _is_active = True
            _event_tracker_arn = t['eventTrackerArn']
            d_resp = personalize.describe_event_tracker(eventTrackerArn = _event_tracker_arn)

            _tracking_id = d_resp['eventTracker']['trackingId']
    
    return _is_active, _event_tracker_arn, _tracking_id

Create the event tracker if it doesn't exist yet.

In [20]:
(exists, tracker_arn, tracking_id) = is_tracker_active(tracker_name)
if not exists:
    response = personalize.create_event_tracker(
        name=tracker_name,
        datasetGroupArn=dg_arn
    )
    print(response['eventTrackerArn'])
    print(response['trackingId'])

    TRACKING_ID = response['trackingId']
else:
    TRACKING_ID = tracking_id

arn:aws:personalize:us-east-1:386102487792:event-tracker/76a6a31f
caad6661-6573-4ebd-81ec-53114d8b78d4


**Definition of important functions:**

Function for defining the user session identifier:

In [23]:
session_dict = {}
def configure_session(user_id):
    # Configure Session
    try:
        _session_ID = session_dict[user_id]
    except:
        session_dict[user_id] = str(uuid.uuid1())
        _session_ID = session_dict[user_id]
    return _session_ID

Function for sending the review with the created event tracker:

In [36]:
def send_new_review(user_id, item_id, timestamp, event_rating, event_verified_purchase, event_type):
      
    session_id = configure_session(user_id)
    
    # Configure Properties:
    event = {
        'itemId': str(item_id),
        'eventRating': event_rating,
        'eventVerifiedPurchase': event_verified_purchase    
    }
    event_json = json.dumps(event)
        
    # Make Call
    personalize_events.put_events(
        trackingId = TRACKING_ID,
        userId     = str(user_id),
        sessionId  = session_id,
        eventList  = [{
            'sentAt': timestamp,
            'eventType': event_type,
            'properties': event_json
            }]
    )

Function for recommending items to a user ID with a given campaign:

In [37]:
def recommend_items(user_id, campaign_arn):
    response = personalize_runtime.get_recommendations(campaignArn=campaign_arn, 
                                                       userId=str(user_id), 
                                                       numResults=10)
    items = response['itemList']
    for item in items:
        print_item(item['itemId'])
    print('')

### Sending new reviews as events:

We will use a sample user:

In [62]:
sample_user = int_expanded_df.sample(1).iloc[0]['USER_ID']
print('Here is a sample user for exercising real time events: ', sample_user)

Here is a sample user for exercising real time events:  47965132


Previous interactions of this user:

In [63]:
show_user_interaction_history(int_expanded_df, sample_user)

Interactions:  1


,USER_ID,ITEM_ID,WHEN
2433478,47965132,A Christmas Story (1983),2014-12-23 00:00:00


Before sending new real-time vents, we will retrieve the Personalize recommendations for this user:

In [32]:
print('Before any real time events, Personalize should recommend:')
if is_campaign_active(hrnn_metadata_hpo_arn):
    print('Next using {}\n'.format(hrnn_metadata_hpo_arn))
    recommend_items(sample_user, hrnn_metadata_hpo_arn)
else:
    print('HRNN-metadata campaign not active: {}'.format(hrnn_metadata_hpo_arn))

Before any real time events, Personalize should recommend:
Next using arn:aws:personalize:us-east-1:386102487792:campaign/video-hrnn-metadata-hpo

Item: Zero Dark Thirty
Item: Life of Pi
Item: Django Unchained
Item: Lincoln
Item: End of Watch
Item: Argo
Item: Beasts of the Southern Wild
Item: Here Comes The Boom
Item: Trouble With The Curve
Item: Taken 2



We will send new reviews with the event tracker:

In [40]:
new_items_reviewed="Pilot"
timestamp=time.time()
send_new_review(sample_user, new_items_reviewed, timestamp, 5, 'Y', 'RATING')

Recommendations after new item review:

In [45]:
print('Now this same user has reviewed {}'.format(new_items_reviewed))
print('Lets see if Personalize picks up on this real time change in intent...\n')

if is_campaign_active(hrnn_metadata_hpo_arn):
    print('Next using {}\n'.format(hrnn_metadata_hpo_arn))
    recommend_items(sample_user, hrnn_metadata_hpo_arn)
else:
    print('HRNN-metadata campaign not active: {}'.format(hrnn_metadata_hpo_arn))

Now this same user has reviewed Pilot
Lets see if Personalize picks up on this real time change in intent...

Next using arn:aws:personalize:us-east-1:386102487792:campaign/video-hrnn-metadata-hpo

Item: Stargate SG-1 Season 3
Item: Doctor Who: Last Christmas
Item: The Man In the High Castle - Season 1
Item: Stargate SG-1 Season 6
Item: Robocop (2014)
Item: Stargate Atlantis Season 2
Item: Stargate SG-1 Season 4
Item: Stargate Atlantis Season 4
Item: Grimm Season 3
Item: Stargate Universe Season 2



We observe that the recommendations have changed after the review made with the event tracker